In [179]:
import numpy as np
import matplotlib.pyplot as plt
import arrow
import datetime
import xarray as xr
import os
import cmocean.cm as cm
import matplotlib.cm as cma
import scipy.interpolate as interp
from matplotlib.collections import PatchCollection
import matplotlib.dates as mpl_dates
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import FormatStrFormatter
import datetime as dt

### Functions

In [135]:
def get_data(datafile):
    saltf = datafile.final_salt
    transpf = datafile.final_transp
    lonf = datafile.final_lon
    latf = datafile.final_lat
    depthf = datafile.final_depth
    sectionf = datafile.final_section
    zf = datafile.final_z
    tempf = datafile.final_temp
    
    return saltf,transpf,lonf,latf,depthf,sectionf,zf,tempf

In [177]:
#Function to extract data from 1 section
def get_secdata(datafile,sectionnum):
    saltf = datafile.final_salt
    transpf = datafile.final_transp
    lonf = datafile.final_lon
    latf = datafile.final_lat
    depthf = datafile.final_depth
    sectionf = datafile.final_section
    zf = datafile.final_z
    tempf = datafile.final_temp
    
    idx = np.where(sectionf==sectionnum)
    transp_sec = transpf[idx]
    lon_sec = lonf[idx]
    depth_sec = depthf[idx]
    zf_sec = zf[idx]
    salt_sec = saltf[idx]
    temp_sec = tempf[idx]
    
    return {'salt':salt_sec,'transport':transp_sec,'lon':lon_sec,'depth':depth_sec,'z':zf_sec,'temp':temp_sec}

In [137]:
def plotTS(axn,var,lonsec,zsec,varsec,lonW,lonE,depi,time,title,cblabel,ylabel,vmin=None,vmax=None):
    
    cmap = cm.deep
    
    m2lon = ((u_lons[lonE]-u_lons[lonW])/(mesh.e2f[0, latgridi, lonW:lonE].sum())).values #degrees/m
    
    arrw,xe,ye,im = axn.hist2d(lonsec/m2lon,df(zsec)[0],weights=varsec,cmap=cmap,bins=[np.array(f_lons[lonW:lonE:1]/m2lon),w_depths[0,0:depi]])
    arrnw,xe2,ye2,im2 = axn.hist2d(lonsec/m2lon,df(zsec)[0],cmap=cmap,bins=[np.array(f_lons[lonW:lonE:1]/m2lon),w_depths[0,0:depi]])
    
    arr3 = arrw/arrnw
    
    X,Y = np.meshgrid(xe,ye)
    
    im = axn.pcolormesh(X,Y,arr3.T,vmin=vmin,vmax=vmax)
    axn.set_xlabel('Longitude (°E)')
    axn.set_title(title)
    axn.invert_yaxis()
    axn.set_xticks(np.linspace(f_lons[lonW]/m2lon,f_lons[lonE-1]/m2lon,4))
    axn.set_xticklabels(['{:.2f}'.format(label) for label in np.linspace(f_lons[lonW],f_lons[lonE-1],4)])
    
    if cblabel == True:
        cb = fig.colorbar(im,ax=axn)
        
        if var == 'temp':
            cb.set_label('Temperature (°C)')
        if var == 'sal':
            cb.set_label('Salinity (g/kg)')
            
    if ylabel == True:
        axn.set_ylabel('Depth (m)')

In [138]:
def get_datat(datafile,section, starttime, endtime):
    final_section = datafile.final_section
    final_transport = datafile.final_transp
    transports = np.sum(np.where(final_section == section, final_transport, 0)) 
    return transports/(endtime-starttime+1)

In [139]:
data = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/for_jan18/for_straight_01jan18_s7t28/ariane_positions_quantitative.nc')
mesh = xr.open_dataset('/home/sallen/MEOPAR/grid/mesh_mask202108.nc')

lonW = 308
lonE = 327
latgridi = 300
latgridf = latgridi-1
fmask = mesh.fmask[0, :, latgridi]
tmask = mesh.tmask[0]
    
lons = data.init_lon[(data.final_section != 0)]
    
f_lons = mesh.glamf[0, latgridi]
u_lons = mesh.glamv[0, latgridi]
w_depths = mesh.gdepw_1d

m2lon = ((u_lons[lonE]-u_lons[lonW])/(mesh.e2f[0, latgridi, lonW:lonE].sum())).values #degrees/m

df = interp.interp1d(mesh.z,mesh.gdepw_1d) # Interpolate to get same binning scheme as SalishSeaCast

### Importing data

In [180]:
##Forwards - Eastern JdF
#Jan
datafas = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/foradm_jan18/for_straight_01jan18_s7t28/ariane_positions_quantitative.nc')
datafds = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/fordec_jan18/for_straight_01jan18_s7t28/ariane_positions_quantitative.nc')
datafhsvs = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forharoVS_jan18/for_straight_01jan18_s7t28/ariane_positions_quantitative.nc')
datafsjvs = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forsjcVS_jan18/back_straight_28jan18_s7t28/ariane_positions_quantitative.nc')
datafrsvs = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forrosVS_jan18/for_straight_01jan18_s7t28/ariane_positions_quantitative.nc')

#July
datafas07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/foradm_jul18/for_straight_01jul18_s7t28/ariane_positions_quantitative.nc')
datafds07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/fordec_jul18/for_straight_01jul18_s7t28/ariane_positions_quantitative.nc')
datafhsvs07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forharoVS_jul18/for_straight_01jul18_s7t28/ariane_positions_quantitative.nc')
datafsjvs07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forsjcVS_jul18/for_straight_01jul18_s7t28/ariane_positions_quantitative.nc')
datafrsvs07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forrosVS_jul18/for_straight_01jul18_s7t28/ariane_positions_quantitative.nc')
datafvs07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/for_jul18/for_straight_01jul18_s7t28/ariane_positions_quantitative.nc')

##Backwards
databvs = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/backVS_jan18/back_straight_28jan18_s7t28/ariane_positions_quantitative.nc')
databas = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/backADM_jan18/back_straight_28jan18_s7t28/ariane_positions_quantitative.nc')
databds = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/backdec_jan18/back_straight_01jan18_s7t28/ariane_positions_quantitative.nc')
databsjvs = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/backsjcVS_jan18/back_straight_28jan18_s7t28/ariane_positions_quantitative.nc')
databrsvs = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/backrosVS_jan18/back_straight_28jan18_s7t28/ariane_positions_quantitative.nc')
databhsvs = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/backharoVS_jan18/back_straight_28jan18_s7t28/ariane_positions_quantitative.nc')

#July
databvs07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/backVS_jul18/back_straight_28jul18_s7t28/ariane_positions_quantitative.nc')
databas07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/backadm_jul18/back_straight_28jul18_s7t28/ariane_positions_quantitative.nc')
databds07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/backdec_jul18/back_straight_28jul18_s7t28/ariane_positions_quantitative.nc')
databsjvs07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/backsjcVS_jul18/back_straight_28jul18_s7t28/ariane_positions_quantitative.nc')
databrsvs07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/backrosVS_jul18/back_straight_28jul18_s7t28/ariane_positions_quantitative.nc')
databhsvs07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/backharoVS_jul18/back_straight_28jul18_s7t28/ariane_positions_quantitative.nc')

##Forwards - SJGI
# Jan
datafg1 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forgulf1_jan18/for_straight_01jan18_s7t28/ariane_positions_quantitative.nc')
datafg2 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forgulf2_jan18/for_straight_01jan18_s7t28/ariane_positions_quantitative.nc')
datafhspr = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forharoPR_jan18/for_straight_01jan18_s7t28/ariane_positions_quantitative.nc')
datafsjpr = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forsjcPR_jan18/for_straight_01jan18_s7t28/ariane_positions_quantitative.nc')
datafrspr = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forrosPR_jan18/for_straight_01jan18_s7t28/ariane_positions_quantitative.nc')
datafpr = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/for_jan18_PR/for_straight_01jan18_s7t28/ariane_positions_quantitative.nc')

# Jul
datafg107 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forgulf1_jul18/for_straight_01jul18_s7t28/ariane_positions_quantitative.nc')
datafg207 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forgulf2_jul18/for_straight_01jul18_s7t28/ariane_positions_quantitative.nc')
datafhspr07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forharoPR_jul18/for_straight_01jul18_s7t28/ariane_positions_quantitative.nc')
datafsjpr07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forsjcPR_jul18/for_straight_01jul18_s7t28/ariane_positions_quantitative.nc')
datafrspr07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forrosPR_jul18/for_straight_01jul18_s7t28/ariane_positions_quantitative.nc')
datafpr07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/forPR_jul18/for_straight_01jul18_s7t28/ariane_positions_quantitative.nc')

##Backwards
# Jan
databg1 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/backgulf1_jan18/back_straight_01jan18_s7t28/ariane_positions_quantitative.nc')
databg2 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/backgulf2_jan18/back_straight_01jan18_s7t28/ariane_positions_quantitative.nc')
databhspr = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/backharoPR_jan18/back_straight_01jan18_s7t28/ariane_positions_quantitative.nc')
databsjpr = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/backsjcPR_jan18/back_straight_01jan18_s7t28/ariane_positions_quantitative.nc')
databrspr = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/backrosPR_jan18/back_straight_01jan18_s7t28/ariane_positions_quantitative.nc')
databpr = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/backPR_jan18/back_straight_28jan18_s7t28/ariane_positions_quantitative.nc')

# Jul
databg107 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/backgulf1_jul18/back_straight_28jul18_s7t28/ariane_positions_quantitative.nc')
databg207 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/backgulf2_jul18/back_straight_28jul18_s7t28/ariane_positions_quantitative.nc')
databhspr07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/backharoPR_jul18/back_straight_01jul18_s7t28/ariane_positions_quantitative.nc')
databsjpr07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/backsjcPR_jul18/back_straight_28jul18_s7t28/ariane_positions_quantitative.nc')
databrspr07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/backrosPR_jul18/back_straight_28jul18_s7t28/ariane_positions_quantitative.nc')
databpr07 = xr.open_dataset('/ocean/cstang/MOAD/analysis-camryn/Ariane/backPR_jul18/back_straight_28jul18_s7t28/ariane_positions_quantitative.nc')

data_dict = {'datafvs':data,'datafas':datafas,'datafds':datafds,'datafrsvs':datafrsvs,'datafsjvs':datafsjvs,'datafhsvs':datafhsvs,
             'datafvs07':datafvs07,'datafas07':datafas07,'datafds07':datafds07,'datafrsvs07':datafrsvs07,'datafsjvs07':datafsjvs07,'datafhsvs07':datafhsvs07,
             'databvs':databvs,'databas':databas,'databds':databds,'databrsvs':databrsvs,'databsjvs':databsjvs,'databhsvs':databhsvs,
             'databvs07':databvs07,'databas07':databas07,'databds07':databds07,'databrsvs07':databrsvs07,'databsjvs07':databsjvs07,'databhsvs07':databhsvs07,
             'datafpr':datafpr,'datafg1':datafg1,'datafg2':datafg2,'datafrspr':datafrspr,'datafsjpr':datafsjpr,'datafhspr':datafhspr,
             'datafpr07':datafpr07,'datafg107':datafg107,'datafg207':datafg207,'datafrspr07':datafrspr07,'datafsjpr07':datafsjpr07,'datafhspr07':datafhspr07,
             'databpr':databpr,'databg1':databg1,'databg2':databg2,'databrspr':databrspr,'databsjpr':databsjpr,'databhspr':databhspr,
             'databpr07':databpr07,'databg107':databg107,'databg207':databg207,'databrspr07':databrspr07,'databsjpr07':databsjpr07,'databhspr07':databhspr07,
}

In [181]:
starttime = 1
endtime = 168

sectionnum = [0,2,3,4,5,6]

var = ['section_transpfvs','section_transpfas','section_transpfds','section_transpfrsvs','section_transpfsjvs','section_transpfhsvs',
       'section_transpfvs07','section_transpfas07','section_transpfds07','section_transpfrsvs07','section_transpfsjvs07','section_transpfhsvs07',
       'section_transpbvs','section_transpbas','section_transpbds','section_transpbrsvs','section_transpbsjvs','section_transpbhsvs',
       'section_transpbvs07','section_transpbas07','section_transpbds07','section_transpbrsvs07','section_transpbsjvs07','section_transpbhsvs07',
       'section_transpfpr','section_transpfg1','section_transpfg2','section_transpfrspr','section_transpfsjpr','section_transpfhspr',
       'section_transpfpr07','section_transpfg107','section_transpfg207','section_transpfrspr07','section_transpfsjpr07','section_transpfhspr07',
       'section_transpbpr','section_transpbg1','section_transpbg2','section_transpbrspr','section_transpbsjpr','section_transpbhspr',
       'section_transpbpr07','section_transpbg107','section_transpbg207','section_transpbrspr07','section_transpbsjpr07','section_transpbhspr07',
       ]
data_name = ['datafvs','datafas','datafds','datafrsvs','datafsjvs','datafhsvs',
             'datafvs07','datafas07','datafds07','datafrsvs07','datafsjvs07','datafhsvs07',
             'databvs','databas','databds','databrsvs','databsjvs','databhsvs',
             'databvs07','databas07','databds07','databrsvs07','databsjvs07','databhsvs07',
             'datafpr','datafg1','datafg2','datafrspr','datafsjpr','datafhspr',
             'datafpr07','datafg107','datafg207','datafrspr07','datafsjpr07','datafhspr07',
             'databpr','databg1','databg2','databrspr','databsjpr','databhspr',
             'databpr07','databg107','databg207','databrspr07','databsjpr07','databhspr07',
             ]

dict_sectiontransp = {}

for i in range(len(var)):
    variable_name = var[i]
    
    if variable_name not in dict_sectiontransp:
        dict_sectiontransp[variable_name] = []
    
    for section in sectionnum:
        dict_sectiontransp[variable_name].append(get_datat(data_dict[data_name[i]],section,starttime,endtime))
    

## Balance - Eastern Juan de Fuca, Jan (F)

In [182]:
leave_vic = np.sum(dict_sectiontransp['section_transpfvs'][1:])
ent_vic = np.sum([dict_sectiontransp['section_transpfhsvs'][5],dict_sectiontransp['section_transpfsjvs'][4],dict_sectiontransp['section_transpfrsvs'][3],dict_sectiontransp['section_transpfds'][2],dict_sectiontransp['section_transpfas'][1]])

leave_adm = np.sum(dict_sectiontransp['section_transpfas'][1:])
ent_adm = np.sum([dict_sectiontransp['section_transpfvs'][1],dict_sectiontransp['section_transpfhsvs'][1],dict_sectiontransp['section_transpfsjvs'][1],dict_sectiontransp['section_transpfrsvs'][1],dict_sectiontransp['section_transpfds'][1]])

leave_haro = np.sum(dict_sectiontransp['section_transpfhsvs'][1:])
ent_haro = np.sum([dict_sectiontransp['section_transpfvs'][5],dict_sectiontransp['section_transpfas'][5],dict_sectiontransp['section_transpfds'][5],dict_sectiontransp['section_transpfrsvs'][5],dict_sectiontransp['section_transpfsjvs'][5]])

leave_ros = np.sum(dict_sectiontransp['section_transpfrsvs'][1:])
ent_ros = np.sum([dict_sectiontransp['section_transpfvs'][3],dict_sectiontransp['section_transpfas'][3],dict_sectiontransp['section_transpfds'][3],dict_sectiontransp['section_transpfsjvs'][3],dict_sectiontransp['section_transpfhsvs'][3]])

leave_sjc = np.sum(dict_sectiontransp['section_transpfsjvs'][1:])
ent_sjc = np.sum([dict_sectiontransp['section_transpfvs'][4],dict_sectiontransp['section_transpfas'][4],dict_sectiontransp['section_transpfds'][4],dict_sectiontransp['section_transpfrsvs'][4],dict_sectiontransp['section_transpfhsvs'][4]])

leave_dec = np.sum(dict_sectiontransp['section_transpfds'][1:])
ent_dec = np.sum([dict_sectiontransp['section_transpfvs'][2],dict_sectiontransp['section_transpfas'][2],dict_sectiontransp['section_transpfrsvs'][2],dict_sectiontransp['section_transpfsjvs'][2],dict_sectiontransp['section_transpfhsvs'][2]])

leave_total = np.sum([leave_haro,leave_vic,leave_ros,leave_sjc,leave_dec,leave_adm])
ent_total = np.sum([ent_haro,ent_vic,ent_adm,ent_dec,ent_ros,ent_sjc])

print('Leaving VicSill:',leave_vic)
print('Leaving Admiralty:',leave_adm)
print('Leaving Haro:',leave_haro)
print('Leaving Rosario:',leave_ros)
print('Leaving San Juan:',leave_sjc)
print('Leaving Deception:',leave_dec)
print('Total:',leave_total,'\n')

print('Entering Victoria:',ent_vic)
print('Entering Admiralty:',ent_adm)
print('Entering Haro:',ent_haro)
print('Entering Rosario:',ent_ros)
print('Entering San Juan:',ent_sjc)
print('Entering Deception:',ent_dec)
print('Total:',ent_total)

Leaving VicSill: 115442.9414089004
Leaving Admiralty: 46320.05326780694
Leaving Haro: 83885.8340332499
Leaving Rosario: 30763.5912555483
Leaving San Juan: 18648.29773326977
Leaving Deception: 2036.5138985505982
Total: 297097.2315973259 

Entering Victoria: 115677.48659706494
Entering Admiralty: 42750.232752231015
Entering Haro: 100828.87593089005
Entering Rosario: 17590.059868435932
Entering San Juan: 18917.95542585463
Entering Deception: 1332.6210228493414
Total: 297097.23159732594


## Balance - Eastern Juan de Fuca, July (F)

In [183]:
leave_vic = np.sum(dict_sectiontransp['section_transpfvs07'][1:])
ent_vic = np.sum([dict_sectiontransp['section_transpfhsvs07'][5],dict_sectiontransp['section_transpfsjvs07'][4],dict_sectiontransp['section_transpfrsvs07'][3],dict_sectiontransp['section_transpfds07'][2],dict_sectiontransp['section_transpfas07'][1]])

leave_adm = np.sum(dict_sectiontransp['section_transpfas07'][1:])
ent_adm = np.sum([dict_sectiontransp['section_transpfvs07'][1],dict_sectiontransp['section_transpfhsvs07'][1],dict_sectiontransp['section_transpfsjvs07'][1],dict_sectiontransp['section_transpfrsvs07'][1],dict_sectiontransp['section_transpfds07'][1]])

leave_haro = np.sum(dict_sectiontransp['section_transpfhsvs07'][1:])
ent_haro = np.sum([dict_sectiontransp['section_transpfvs07'][5],dict_sectiontransp['section_transpfas07'][5],dict_sectiontransp['section_transpfds07'][5],dict_sectiontransp['section_transpfrsvs07'][5],dict_sectiontransp['section_transpfsjvs07'][5]])

leave_ros = np.sum(dict_sectiontransp['section_transpfrsvs07'][1:])
ent_ros = np.sum([dict_sectiontransp['section_transpfvs07'][3],dict_sectiontransp['section_transpfas07'][3],dict_sectiontransp['section_transpfds07'][3],dict_sectiontransp['section_transpfsjvs07'][3],dict_sectiontransp['section_transpfhsvs07'][3]])

leave_sjc = np.sum(dict_sectiontransp['section_transpfsjvs07'][1:])
ent_sjc = np.sum([dict_sectiontransp['section_transpfvs07'][4],dict_sectiontransp['section_transpfas07'][4],dict_sectiontransp['section_transpfds07'][4],dict_sectiontransp['section_transpfrsvs07'][4],dict_sectiontransp['section_transpfhsvs07'][4]])

leave_dec = np.sum(dict_sectiontransp['section_transpfds07'][1:])
ent_dec = np.sum([dict_sectiontransp['section_transpfvs07'][2],dict_sectiontransp['section_transpfas07'][2],dict_sectiontransp['section_transpfrsvs07'][2],dict_sectiontransp['section_transpfsjvs07'][2],dict_sectiontransp['section_transpfhsvs07'][2]])

leave_total = np.sum([leave_haro,leave_vic,leave_ros,leave_sjc,leave_dec,leave_adm])
ent_total = np.sum([ent_haro,ent_vic,ent_adm,ent_dec,ent_ros,ent_sjc])

print('Leaving VicSill:',leave_vic)
print('Leaving Admiralty:',leave_adm)
print('Leaving Haro:',leave_haro)
print('Leaving Rosario:',leave_ros)
print('Leaving San Juan:',leave_sjc)
print('Leaving Deception:',leave_dec)
print('Total:',leave_total,'\n')

print('Entering Victoria:',ent_vic)
print('Entering Admiralty:',ent_adm)
print('Entering Haro:',ent_haro)
print('Entering Rosario:',ent_ros)
print('Entering San Juan:',ent_sjc)
print('Entering Deception:',ent_dec)
print('Total:',ent_total)

Leaving VicSill: 143789.43111006363
Leaving Admiralty: 39007.37415168765
Leaving Haro: 82594.40970816233
Leaving Rosario: 55141.46214215933
Leaving San Juan: 21489.6065426656
Leaving Deception: 1470.9167118085136
Total: 343493.20036654704 

Entering Victoria: 161743.18112842017
Entering Admiralty: 38331.41840890986
Entering Haro: 115976.44535501253
Entering Rosario: 14912.80799921974
Entering San Juan: 11778.822581549859
Entering Deception: 750.5248934348705
Total: 343493.2003665471


## Balance - Eastern Juan de Fuca, July (B)

In [163]:
leave_vic = np.sum(dict_sectiontransp['section_transpbvs'][1:])
ent_vic = np.sum([dict_sectiontransp['section_transpbhsvs'][5],dict_sectiontransp['section_transpbsjvs'][4],dict_sectiontransp['section_transpbrsvs'][3],dict_sectiontransp['section_transpbds'][2],dict_sectiontransp['section_transpbas'][1]])

leave_adm = np.sum(dict_sectiontransp['section_transpbas'][1:])
ent_adm = np.sum([dict_sectiontransp['section_transpbvs'][1],dict_sectiontransp['section_transpbhsvs'][1],dict_sectiontransp['section_transpbsjvs'][1],dict_sectiontransp['section_transpbrsvs'][1],dict_sectiontransp['section_transpbds'][1]])

leave_haro = np.sum(dict_sectiontransp['section_transpbhsvs'][1:])
ent_haro = np.sum([dict_sectiontransp['section_transpbvs'][5],dict_sectiontransp['section_transpbas'][5],dict_sectiontransp['section_transpbds'][5],dict_sectiontransp['section_transpbrsvs'][5],dict_sectiontransp['section_transpbsjvs'][5]])

leave_ros = np.sum(dict_sectiontransp['section_transpbrsvs'][1:])
ent_ros = np.sum([dict_sectiontransp['section_transpbvs'][3],dict_sectiontransp['section_transpbas'][3],dict_sectiontransp['section_transpbds'][3],dict_sectiontransp['section_transpbsjvs'][3],dict_sectiontransp['section_transpbhsvs'][3]])

leave_sjc = np.sum(dict_sectiontransp['section_transpbsjvs'][1:])
ent_sjc = np.sum([dict_sectiontransp['section_transpbvs'][4],dict_sectiontransp['section_transpbas'][4],dict_sectiontransp['section_transpbds'][4],dict_sectiontransp['section_transpbrsvs'][4],dict_sectiontransp['section_transpbhsvs'][4]])

leave_dec = np.sum(dict_sectiontransp['section_transpbds'][1:])
ent_dec = np.sum([dict_sectiontransp['section_transpbvs'][2],dict_sectiontransp['section_transpbas'][2],dict_sectiontransp['section_transpbrsvs'][2],dict_sectiontransp['section_transpbsjvs'][2],dict_sectiontransp['section_transpbhsvs'][2]])

leave_total = np.sum([leave_haro,leave_vic,leave_ros,leave_sjc,leave_dec,leave_adm])
ent_total = np.sum([ent_haro,ent_vic,ent_adm,ent_dec,ent_ros,ent_sjc])

print('Leaving VicSill:',leave_vic)
print('Leaving Admiralty:',leave_adm)
print('Leaving Haro:',leave_haro)
print('Leaving Rosario:',leave_ros)
print('Leaving San Juan:',leave_sjc)
print('Leaving Deception:',leave_dec)
print('Total:',leave_total,'\n')

print('Entering Victoria:',ent_vic)
print('Entering Admiralty:',ent_adm)
print('Entering Haro:',ent_haro)
print('Entering Rosario:',ent_ros)
print('Entering San Juan:',ent_sjc)
print('Entering Deception:',ent_dec)
print('Total:',ent_total)

Leaving VicSill: 69775.01367985812
Leaving Admiralty: 33274.63536649126
Leaving Haro: 52429.9331715027
Leaving Rosario: 8982.600916157924
Leaving San Juan: 13603.718251741293
Leaving Deception: 1894.4070192909705
Total: 179960.30840504222 

Entering Victoria: 64195.62377027394
Entering Admiralty: 28874.556372477015
Entering Haro: 56909.77481054612
Entering Rosario: 16402.42651761673
Entering San Juan: 13295.025179388338
Entering Deception: 282.9017547401383
Total: 179960.30840504228


## Balance - SJGI, Jan (F)

In [155]:
leave_haro = np.sum(dict_sectiontransp['section_transpfhspr'][1:])
leave_sjc = np.sum(dict_sectiontransp['section_transpfsjpr'][1:])
leave_ros = np.sum(dict_sectiontransp['section_transpfrspr'][1:])
leave_pr = np.sum(dict_sectiontransp['section_transpfpr'][1:])
leave_g1 = np.sum(dict_sectiontransp['section_transpfg1'][1:])
leave_g2 = np.sum(dict_sectiontransp['section_transpfg2'][1:])
leave_total = np.sum([leave_haro,leave_sjc,leave_ros,leave_pr,leave_g1,leave_g2])

ent_haro = np.sum([dict_sectiontransp['section_transpfsjpr'][5],dict_sectiontransp['section_transpfrspr'][5],dict_sectiontransp['section_transpfpr'][5],dict_sectiontransp['section_transpfg1'][5],dict_sectiontransp['section_transpfg2'][5]])
ent_pr = np.sum([dict_sectiontransp['section_transpfhspr'][5],dict_sectiontransp['section_transpfsjpr'][4],dict_sectiontransp['section_transpfrspr'][3],dict_sectiontransp['section_transpfg1'][1],dict_sectiontransp['section_transpfg2'][2]])
ent_sjc = np.sum([dict_sectiontransp['section_transpfhspr'][4],dict_sectiontransp['section_transpfrspr'][4],dict_sectiontransp['section_transpfpr'][4],dict_sectiontransp['section_transpfg1'][4],dict_sectiontransp['section_transpfg2'][4]])
ent_ros = np.sum([dict_sectiontransp['section_transpfsjpr'][3],dict_sectiontransp['section_transpfhspr'][3],dict_sectiontransp['section_transpfpr'][3],dict_sectiontransp['section_transpfg1'][3],dict_sectiontransp['section_transpfg2'][3]])
ent_g1 = np.sum([dict_sectiontransp['section_transpfsjpr'][1],dict_sectiontransp['section_transpfrspr'][1],dict_sectiontransp['section_transpfpr'][1],dict_sectiontransp['section_transpfhspr'][1],dict_sectiontransp['section_transpfg2'][1]])
ent_g2 = np.sum([dict_sectiontransp['section_transpfsjpr'][2],dict_sectiontransp['section_transpfrspr'][2],dict_sectiontransp['section_transpfpr'][2],dict_sectiontransp['section_transpfg1'][2],dict_sectiontransp['section_transpfhspr'][2]])

ent_total = np.sum([ent_haro,ent_pr,ent_ros,ent_sjc,ent_g1,ent_g2])

print('Leaving Haro:',leave_haro)
print('Leaving SanJuan: ',leave_sjc)
print('Leaving Rosario:',leave_ros)
print('Leaving PointRob:',leave_pr)
print('Leaving Gulf1:',leave_g1)
print('Leaving Gulf2:',leave_g2)
print('Total',leave_total,'\n')

print('Entering Haro:',ent_haro)
print('Entering SanJuan: ',ent_sjc)
print('Entering Rosario:',ent_ros)
print('Entering PointRob:',ent_pr)
print('Entering Gulf1:',ent_g1)
print('Entering Gulf2:',ent_g2)
print('Total',ent_total)

Leaving Haro: 46570.24167264905
Leaving SanJuan:  5550.4500176716865
Leaving Rosario: 4623.476834830786
Leaving PointRob: 35542.598489668446
Leaving Gulf1: 10694.064688829832
Leaving Gulf2: 3830.6871710463574
Total 106811.51887469615 

Entering Haro: 33242.74294482853
Entering SanJuan:  7059.222178079723
Entering Rosario: 17614.23125150292
Entering PointRob: 36272.978528579
Entering Gulf1: 8672.954445048696
Entering Gulf2: 3949.389526657297
Total 106811.51887469616


## Balance - SJGI, July (F)

In [156]:
leave_haro = np.sum(dict_sectiontransp['section_transpfhspr07'][1:])
leave_sjc = np.sum(dict_sectiontransp['section_transpfsjpr07'][1:])
leave_ros = np.sum(dict_sectiontransp['section_transpfrspr07'][1:])
leave_pr = np.sum(dict_sectiontransp['section_transpfpr07'][1:])
leave_g1 = np.sum(dict_sectiontransp['section_transpfg107'][1:])
leave_g2 = np.sum(dict_sectiontransp['section_transpfg207'][1:])
leave_total = np.sum([leave_haro,leave_sjc,leave_ros,leave_pr,leave_g1,leave_g2])

ent_haro = np.sum([dict_sectiontransp['section_transpfsjpr07'][5],dict_sectiontransp['section_transpfrspr07'][5],dict_sectiontransp['section_transpfpr07'][5],dict_sectiontransp['section_transpfg107'][5],dict_sectiontransp['section_transpfg207'][5]])
ent_pr = np.sum([dict_sectiontransp['section_transpfhspr07'][5],dict_sectiontransp['section_transpfsjpr07'][4],dict_sectiontransp['section_transpfrspr07'][3],dict_sectiontransp['section_transpfg107'][1],dict_sectiontransp['section_transpfg207'][2]])
ent_sjc = np.sum([dict_sectiontransp['section_transpfhspr07'][4],dict_sectiontransp['section_transpfrspr07'][4],dict_sectiontransp['section_transpfpr07'][4],dict_sectiontransp['section_transpfg107'][4],dict_sectiontransp['section_transpfg207'][4]])
ent_ros = np.sum([dict_sectiontransp['section_transpfsjpr07'][3],dict_sectiontransp['section_transpfhspr07'][3],dict_sectiontransp['section_transpfpr07'][3],dict_sectiontransp['section_transpfg107'][3],dict_sectiontransp['section_transpfg207'][3]])
ent_g1 = np.sum([dict_sectiontransp['section_transpfsjpr07'][1],dict_sectiontransp['section_transpfrspr07'][1],dict_sectiontransp['section_transpfpr07'][1],dict_sectiontransp['section_transpfhspr07'][1],dict_sectiontransp['section_transpfg207'][1]])
ent_g2 = np.sum([dict_sectiontransp['section_transpfsjpr07'][2],dict_sectiontransp['section_transpfrspr07'][2],dict_sectiontransp['section_transpfpr07'][2],dict_sectiontransp['section_transpfg107'][2],dict_sectiontransp['section_transpfhspr07'][2]])

ent_total = np.sum([ent_haro,ent_pr,ent_ros,ent_sjc,ent_g1,ent_g2])

print('Leaving Haro:',leave_haro)
print('Leaving SanJuan: ',leave_sjc)
print('Leaving Rosario:',leave_ros)
print('Leaving PointRob:',leave_pr)
print('Leaving Gulf1:',leave_g1)
print('Leaving Gulf2:',leave_g2)
print('Total',leave_total,'\n')

print('Entering Haro:',ent_haro)
print('Entering SanJuan: ',ent_sjc)
print('Entering Rosario:',ent_ros)
print('Entering PointRob:',ent_pr)
print('Entering Gulf1:',ent_g1)
print('Entering Gulf2:',ent_g2)
print('Total',ent_total)

Leaving Haro: 73549.9920240067
Leaving SanJuan:  2965.7887606488803
Leaving Rosario: 270.3993197320528
Leaving PointRob: 55291.629419941775
Leaving Gulf1: 9602.536076155557
Leaving Gulf2: 3387.4114142240846
Total 145067.75701470903 

Entering Haro: 37772.173063457034
Entering SanJuan:  10842.133387725988
Entering Rosario: 31249.15103220599
Entering PointRob: 54367.46843558952
Entering Gulf1: 8734.226151764347
Entering Gulf2: 2102.60494396616
Total 145067.75701470903


In [185]:
print(dict_sectiontransp['section_transpfas'])
print(dict_sectiontransp['section_transpbrsvs'])

[101992.0359021202, 31376.8941213031, 13.456059159455611, 2470.603053432186, 1859.0735586814149, 10600.026475230792]
[71463.43953919351, 3467.007507138596, 1659.1203502073604, 3197.3575903872734, 1555.2649157938893, 702.2368033838187]
